# 1

In [2]:
import mariadb
import sys
import pandas as pd


def import_file_into_db_table(filename, database, user, passwd, tableName,
createTableStatement = "", overwrite = False):
    try:
        connection = mariadb.connect(
        user=user,
        password=passwd, # or insert string here
        host="127.0.0.1",
        port=3306,
        database=database
        )
    except mariadb.Error as error:
        print(f'Error connecting to MariaDB: {error}')
        sys.exit(1)

    cursor = connection.cursor()

    


    table_exists = False
    try:
        cursor.execute(f"""DESCRIBE {tableName};""")
        table_exists = True
    except:
        pass

    if(table_exists):
        if(overwrite):
            cursor.execute(f"""LOAD DATA INFILE '{filename}'
                                REPLACE
                                INTO TABLE {tableName}
                                FIELDS TERMINATED BY ';'
                                ENCLOSED BY '"'
                                IGNORE 1 ROWS;""")
        else:
            print('Could not write values into table, because table already exists')
            sys.exit(1)
    else:

        try:
            cursor.execute(createTableStatement)
        except:
            print("createTableStatement failed")
            sys.exit(1)

        cursor.execute(f"""LOAD DATA INFILE '{filename}'
                            INTO TABLE {tableName}
                            FIELDS TERMINATED BY ';'
                            ENCLOSED BY '"'
                            IGNORE 1 ROWS;""")

    #cursor.fetchall()

    # check if rows match
    df = pd.read_csv(filename, delimiter=";")
    cursor.execute(f"SELECT COUNT(*) FROM {tableName};")
    if len(df.index) == cursor.fetchall()[0][0]:
        print("Length of table matches the CSV file")
    else:
        print("Length of table does not match the CSV file")


# 2

In [9]:
import mariadb
import sys
import pandas as pd
import itertools

def join_tables(database, user, passwd, tables, attributes = []):
    try:
        connection = mariadb.connect(
        user=user,
        password=passwd, # or insert string here
        host="127.0.0.1",
        port=3306,
        database=database
        )
    except mariadb.Error as error:
        print(f'Error connecting to MariaDB: {error}')
        sys.exit(1)

    cursor = connection.cursor()
    
    if not attributes:
        # Natural Join
        column_lister = []
        for table in tables:
            tmp_column_set = set()
            cursor.execute(f"DESCRIBE {table};")
            for x in cursor.fetchall():
                tmp_column_set.add(x[0])
                print(x[0])
            column_lister.append(tmp_column_set)
        
        # Schnitt der Columns
        current_list = column_lister[0]
        for col in column_lister[1:]:
            current_list = list(set(current_list) & col)
        
        attributes = current_list


    from_tables = "FROM " + ', '.join(str(x) for x in tables)

    where = "WHERE "

    # 2-elementige Teilmengen
    teilm = list(itertools.combinations(tables, 2))
    

    for y in attributes:
        for t in teilm:
            where += f"{t[0]}.{y}={t[1]}.{y} AND "

    where = where[:-5] + ";"

    expr = f"""SELECT *
                {from_tables}
                {where}"""

    cursor.execute(expr)
    print(cursor.fetchall())


#joins = ["school", "sex", "age", "address", "famsize", "Pstatus", "Medu", "Fedu", "Mjob", "Fjob", "reason", "nursery", "internet"]
#join_tables(database="db2022", user="root", passwd="password", tables = ["student_mat", "student_por"], attributes=joins)

# 3

In [10]:
# a
createTableStatement = ""
with open("createTableStatement.sql") as f:
    createTableStatement = f.read()

createTableStatement2 = ""
with open("createTableStatement2.sql") as f:
    createTableStatement2 = f.read()

import_file_into_db_table(filename="/tmp/student-mat.csv", database="db2022", user="root", passwd="password", tableName="student_mat", createTableStatement=createTableStatement)
import_file_into_db_table(filename="/tmp/student-por.csv", database="db2022", user="root", passwd="password", tableName="student_por", createTableStatement=createTableStatement2)

# b
joins = ["school", "sex", "age", "address", "famsize", "Pstatus", "Medu", "Fedu", "Mjob", "Fjob", "reason", "nursery", "internet"]
join_tables(database="db2022", user="root", passwd="password", tables = ["student_mat", "student_por"], attributes=joins)

# c
try:
    connection = mariadb.connect(
    user="root",
    password="password", # or insert string here
    host="127.0.0.1",
    port=3306,
    database="db2022"
    )
except mariadb.Error as error:
    print(f'Error connecting to MariaDB: {error}')
    sys.exit(1)

cursor = connection.cursor()
cursor.execute("DROP VIEW if exists v;")
cursor.execute("""CREATE VIEW v as (
SELECT	student_mat.school,
        student_mat.sex,
        student_mat.age,
        student_mat.address,
        student_mat.famsize,
        student_mat.Pstatus,
        student_mat.Medu,
        student_mat.Fedu,
        student_mat.Mjob,
        student_mat.Fjob,
        student_mat.reason,
        student_mat.nursery,
        student_mat.internet,
        student_mat.guardian as student_matguardian, 
        student_mat.traveltime as student_mattraveltime, 
        student_mat.studytime as student_matstudytime, 
        student_mat.failures as student_matfailures, 
        student_mat.schoolsup as student_matschoolsup, 
        student_mat.famsup as student_matfamsup, 
        student_mat.paid as student_matpaid, 
        student_mat.activities as student_matactivities, 
        student_mat.higher as student_mathigher, 
        student_mat.romantic as student_matromantic, 
        student_mat.famrel as student_matfamrel, 
        student_mat.freetime as student_matfreetime, 
        student_mat.goout as student_matgoout, 
        student_mat.Dalc as student_matDalc, 
        student_mat.Walc as student_matWalc, 
        student_mat.health as student_mathealth, 
        student_mat.absences as student_matabsences, 
        student_mat.G1 as student_matG1, 
        student_mat.G2 as student_matG2, 
        student_mat.G3 as student_matG3, 
        student_por.guardian as student_porguardian, 
        student_por.traveltime as student_portraveltime, 
        student_por.studytime as student_porstudytime, 
        student_por.failures as student_porfailures, 
        student_por.schoolsup as student_porschoolsup, 
        student_por.famsup as student_porfamsup, 
        student_por.paid as student_porpaid, 
        student_por.activities as student_poractivities, 
        student_por.higher as student_porhigher, 
        student_por.romantic as student_porromantic, 
        student_por.famrel as student_porfamrel, 
        student_por.freetime as student_porfreetime, 
        student_por.goout as student_porgoout, 
        student_por.Dalc as student_porDalc, 
        student_por.Walc as student_porWalc, 
        student_por.health as student_porhealth, 
        student_por.absences as student_porabsences, 
        student_por.G1 as student_porG1, 
        student_por.G2 as student_porG2, 
        student_por.G3 as student_porG3
  		FROM student_mat, student_por
		WHERE student_mat.school=student_por.school AND student_mat.sex=student_por.sex AND student_mat.age=student_por.age AND student_mat.address=student_por.address AND student_mat.famsize=student_por.famsize AND student_mat.Pstatus=student_por.Pstatus AND student_mat.Medu=student_por.Medu AND student_mat.Fedu=student_por.Fedu AND student_mat.Mjob=student_por.Mjob AND student_mat.Fjob=student_por.Fjob AND student_mat.reason=student_por.reason AND student_mat.nursery=student_por.nursery AND student_mat.internet=student_por.internet
);""")
cursor.execute("SELECT * from v;")

print(cursor.fetchmany(5))

# d

# 1
cursor.execute("SELECT COUNT(*) as Anzahl, GREATEST(MAX(G1), MAX(G2), MAX(G3)) as Best, (MAX(G1) + AVG(G2) + AVG(G3)) / 3 as Avg FROM student_mat;")
print(cursor.fetchall())

cursor.execute("SELECT COUNT(*) as Anzahl, GREATEST(MAX(G1), MAX(G2), MAX(G3)) as Best, (MAX(G1) + AVG(G2) + AVG(G3)) / 3 as Avg FROM student_por;")
print(cursor.fetchall())

cursor.execute("SELECT  COUNT(*) as Anzahl, GREATEST(MAX(student_matG1), MAX(student_matG2), MAX(student_matG3), MAX(student_porG1), MAX(student_porG2), MAX(student_porG3)) as Best, (AVG(student_matG1) + AVG(student_matG2) + AVG(student_matG3) + AVG(student_porG1) + AVG(student_porG2) + AVG(student_porG3)) / 3 as Avg FROM v;")
print(cursor.fetchall())

# 2
cursor.execute("SELECT GREATEST(G1, G2, G3) as Best, (G1 + G2 + G3) / 3 as AvgG FROM student_mat;")
print(cursor.fetchall())

cursor.execute("SELECT GREATEST(G1, G2, G3) as Best, (G1 + G2 + G3) / 3 as AvgG FROM student_por;")
print(cursor.fetchall())

cursor.execute("SELECT GREATEST(student_matG1, student_matG2, student_matG3, student_porG1, student_porG2, student_porG3) as Best, (student_matG1 + student_matG2 + student_matG3 + student_porG1 + student_porG2 + student_porG3) / 6 as AvgG FROM v;")
print(cursor.fetchall())


[('GP', 'F', 18, 'U', 'GT3', 'A', 4, 4, 'at_home', 'teacher', 'course', 'mother', '2', '2', 0, 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 4, 3, 4, 1, 1, 3, 6, 5, 6, 6, 'GP', 'F', 18, 'U', 'GT3', 'A', 4, 4, 'at_home', 'teacher', 'course', 'mother', '2', '2', 0, 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 4, 3, 4, 1, 1, 3, 4, 0, 11, 11), ('GP', 'F', 17, 'U', 'GT3', 'T', 1, 1, 'at_home', 'other', 'course', 'father', '1', '2', 0, 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 5, 3, 3, 1, 1, 3, 4, 5, 5, 6, 'GP', 'F', 17, 'U', 'GT3', 'T', 1, 1, 'at_home', 'other', 'course', 'father', '1', '2', 0, 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 5, 3, 3, 1, 1, 3, 2, 9, 11, 11), ('GP', 'F', 15, 'U', 'LE3', 'T', 1, 1, 'at_home', 'other', 'other', 'mother', '1', '2', 3, 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 4, 3, 2, 2, 3, 3, 10, 7, 8, 10, 'GP', 'F', 15, 'U', 'LE3', 'T', 1, 1, 'at_home', 'other', 'other', 'mother', '1', '2', 0, 'yes', 'no', 'no', 'no', 'yes', '